In [1]:
import numpy as np

from my_stuff import MufexKeys
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.base import backtest_df_only
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb

mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex.get_candles_df(symbol=symbol, timeframe='5m', candles_to_dl=2000)

It took 00 mins and 04 seconds to download the candles


In [3]:
mufex.exchange_settings

In [4]:
candles

,open,high,low,close
timestamp,,,,
2023-09-30 00:40:00,26914.0,26915.0,26910.3,26914.0
2023-09-30 00:45:00,26914.0,26929.8,26914.0,26921.7
2023-09-30 00:50:00,26921.7,26924.2,26914.2,26921.3
2023-09-30 00:55:00,26921.3,26934.6,26921.2,26922.3
2023-09-30 01:00:00,26922.3,26926.0,26911.3,26918.6
...,...,...,...,...
2023-10-06 22:55:00,27999.9,28004.5,27953.5,27973.0
2023-10-06 23:00:00,27973.0,27973.0,27924.5,27942.1
2023-10-06 23:05:00,27942.1,27943.7,27902.7,27910.2


In [5]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01]) / 100,
    sl_based_on_lookback=np.array([50]),
    risk_reward=np.array([2.0, 5.0]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitStrategyType.Provided]),
    max_equity_risk_pct=np.array([4]) / 100,
    long_or_short=np.array([LongOrShortType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([0.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    num_candles=np.array([50, 150, 250]),
)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [14]:
strategy = Strategy(
    candle_processing_mode=CandleProcessingType.RegularBacktest,
    candles=candles,
)
candles_reg = backtest_df_only(
    starting_equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 6
Total combinations of settings to test: 24

Total candles: 2,000
Total candles to test: 48,000


In [16]:
candles_reg.sort_values(['ind_set_idx','or_set_idx']).iloc[:]

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
8,0,0,384.0,-511.415493,13.28,-0.956529,-836.4447,163.554900
12,0,1,363.0,-403.405471,14.60,-0.956781,-801.3523,198.647027
0,0,2,333.0,-375.691855,15.62,-0.946319,-789.7792,210.220122
9,0,3,384.0,-511.415493,13.28,-0.956529,-836.4447,163.554900
13,0,4,363.0,-403.405471,14.60,-0.956781,-801.3523,198.647027
1,0,5,333.0,-375.691855,15.62,-0.946319,-789.7792,210.220122
20,1,0,163.0,-291.318650,25.15,-0.959263,-744.4538,255.546215
16,1,1,154.0,-256.732409,25.32,-0.957139,-719.6777,280.322161
4,1,2,145.0,-253.687031,26.90,-0.951002,-717.2643,282.735841
21,1,3,163.0,-291.318650,25.15,-0.959263,-744.4538,255.546215


In [29]:
strategy = Strategy(
    candle_processing_mode=CandleProcessingType.CandleBacktest,
    candles=candles,
)
candle_backtest = backtest_df_only(
    starting_equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 6
Total combinations of settings to test: 24

Total candles: 2,000
Total candles to test: 48,000


KeyboardInterrupt: 

In [28]:
candle_backtest.sort_values(['ind_set_idx','or_set_idx'])

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
18,0,0,71.0,-143.747977,16.90,-0.931046,-589.7401,410.259815
10,0,1,70.0,-120.564572,17.14,-0.925525,-546.6180,453.381970
8,0,2,69.0,-144.655404,15.94,-0.925433,-591.2617,408.738162
19,0,3,71.0,-143.747977,16.90,-0.931046,-589.7401,410.259815
11,0,4,70.0,-120.564572,17.14,-0.925525,-546.6180,453.381970
9,0,5,69.0,-144.655404,15.94,-0.925433,-591.2617,408.738162
6,1,0,71.0,-134.204805,16.90,-0.925324,-573.0233,426.976722
2,1,1,70.0,-123.659061,17.14,-0.923203,-552.8909,447.109094
0,1,2,69.0,-124.427721,15.94,-0.918157,-554.4222,445.577755
7,1,3,71.0,-134.204805,16.90,-0.925324,-573.0233,426.976722
